In [1]:
import sys
sys.path.append("/home/hugo/projetos-doutorado/mimo_emb_fts/src/")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from embfts.models.MimoNonStationaryFtsPca import MimoNonStationaryFtsPca
from pyFTS.models.nonstationary import nsfts
from pyFTS.benchmarks import Measures
from pyFTS.benchmarks import Measures
import matplotlib.pyplot as plt
from pyFTS.common import Util
import datetime
import statistics
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
import math


np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
def sample_first_prows(data, perc):
    return data.head(int(len(data)*(perc)))

In [3]:
def cal_nrmse(rmse, y):
    x = max(y)-min(y)
    return (rmse/x)

In [4]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame) 
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

In [5]:
# convert series to supervised learning
def series_to_supervised_mimo(data, n_in, n_out, dropnan=True):
	"""
	Frame a time series as a supervised learning dataset.
	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		n_out: Number of observations as output (y).
		dropnan: Boolean whether or not to drop rows with NaN values.
	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [(df.columns[j]+'(t-%d)' % (i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [(df.columns[j]+'(t)') for j in range(n_vars)]
		else:
			names += [(df.columns[j]+'%d(t+%d)' % (j, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [9]:
df = pd.read_csv('/home/hugo/projetos-doutorado/mimo_emb_fts/data/HomeC.csv', sep=',')
data = df.drop(labels=['icon','summary','cloudCover', 'House overall [kW]', 'time'], axis=1)
data = data.loc[0: : 10]
data.dropna(inplace=True)
data = clean_dataset(data)
data.head()

/home/hugo/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (0,27) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,use [kW],gen [kW],Dishwasher [kW],Furnace 1 [kW],Furnace 2 [kW],Home office [kW],Fridge [kW],Wine cellar [kW],Garage door [kW],Kitchen 12 [kW],...,temperature,humidity,visibility,apparentTemperature,pressure,windSpeed,windBearing,precipIntensity,dewPoint,precipProbability
0,0.932833,0.003483,0.000033,0.020700,0.061917,0.442633,0.124150,0.006983,0.013083,0.000417,...,36.14,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0
10,1.585083,0.003417,0.000050,0.022100,0.678733,0.440200,0.121450,0.007433,0.013583,0.000350,...,36.14,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0
20,0.533817,0.003450,0.000000,0.020633,0.062967,0.270033,0.004950,0.007033,0.013117,0.000750,...,36.14,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0
30,1.190950,0.003083,0.000217,0.387400,0.061883,0.256867,0.004933,0.101783,0.012533,0.000700,...,36.14,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0
40,1.582083,0.003067,0.000050,0.022050,0.681717,0.242433,0.005550,0.122400,0.013550,0.000567,...,36.14,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0


In [10]:
# data = data.iloc[0:50388,:]
# data.tail()


In [13]:
# frame as supervised learning
reframed = series_to_supervised_mimo(data, 1, 1)
reframed

,use [kW](t-1),gen [kW](t-1),Dishwasher [kW](t-1),Furnace 1 [kW](t-1),Furnace 2 [kW](t-1),Home office [kW](t-1),Fridge [kW](t-1),Wine cellar [kW](t-1),Garage door [kW](t-1),Kitchen 12 [kW](t-1),...,temperature(t),humidity(t),visibility(t),apparentTemperature(t),pressure(t),windSpeed(t),windBearing(t),precipIntensity(t),dewPoint(t),precipProbability(t)
10,0.932833,0.003483,0.000033,0.020700,0.061917,0.442633,0.124150,0.006983,0.013083,0.000417,...,36.14,0.62,10.00,29.26,1016.91,9.18,282.0,0.0000,24.40,0.00
20,1.585083,0.003417,0.000050,0.022100,0.678733,0.440200,0.121450,0.007433,0.013583,0.000350,...,36.14,0.62,10.00,29.26,1016.91,9.18,282.0,0.0000,24.40,0.00
30,0.533817,0.003450,0.000000,0.020633,0.062967,0.270033,0.004950,0.007033,0.013117,0.000750,...,36.14,0.62,10.00,29.26,1016.91,9.18,282.0,0.0000,24.40,0.00
40,1.190950,0.003083,0.000217,0.387400,0.061883,0.256867,0.004933,0.101783,0.012533,0.000700,...,36.14,0.62,10.00,29.26,1016.91,9.18,282.0,0.0000,24.40,0.00
50,1.582083,0.003067,0.000050,0.022050,0.681717,0.242433,0.005550,0.122400,0.013550,0.000567,...,36.14,0.62,10.00,29.26,1016.91,9.18,282.0,0.0000,24.40,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503860,1.068350,0.004183,0.000100,0.021550,0.639333,0.041650,0.005350,0.020400,0.013450,0.000567,...,35.12,0.86,8.74,29.45,1011.49,6.72,186.0,0.0101,31.27,0.51
503870,1.621417,0.004183,0.000033,0.441950,0.645467,0.041767,0.129317,0.008067,0.012883,0.000317,...,35.12,0.86,8.74,29.45,1011.49,6.72,186.0,0.0101,31.27,0.51
503880,0.896250,0.004150,0.000233,0.496633,0.063000,0.042317,0.123800,0.007850,0.012433,0.000467,...,35.12,0.86,8.74,29.45,1011.49,6.72,186.0,0.0101,31.27,0.51
503890,1.311617,0.004167,0.000017,0.359167,0.645433,0.041917,0.005133,0.008167,0.013083,0.000650,...,35.12,0.86,8.74,29.45,1011.49,6.72,186.0,0.0101,31.27,0.51


In [14]:
cols = reframed.columns
cols

Index(['use [kW](t-1)', 'gen [kW](t-1)', 'Dishwasher [kW](t-1)',
       'Furnace 1 [kW](t-1)', 'Furnace 2 [kW](t-1)', 'Home office [kW](t-1)',
       'Fridge [kW](t-1)', 'Wine cellar [kW](t-1)', 'Garage door [kW](t-1)',
       'Kitchen 12 [kW](t-1)', 'Kitchen 14 [kW](t-1)', 'Kitchen 38 [kW](t-1)',
       'Barn [kW](t-1)', 'Well [kW](t-1)', 'Microwave [kW](t-1)',
       'Living room [kW](t-1)', 'Solar [kW](t-1)', 'temperature(t-1)',
       'humidity(t-1)', 'visibility(t-1)', 'apparentTemperature(t-1)',
       'pressure(t-1)', 'windSpeed(t-1)', 'windBearing(t-1)',
       'precipIntensity(t-1)', 'dewPoint(t-1)', 'precipProbability(t-1)',
       'use [kW](t)', 'gen [kW](t)', 'Dishwasher [kW](t)', 'Furnace 1 [kW](t)',
       'Furnace 2 [kW](t)', 'Home office [kW](t)', 'Fridge [kW](t)',
       'Wine cellar [kW](t)', 'Garage door [kW](t)', 'Kitchen 12 [kW](t)',
       'Kitchen 14 [kW](t)', 'Kitchen 38 [kW](t)', 'Barn [kW](t)',
       'Well [kW](t)', 'Microwave [kW](t)', 'Living room [kW](t)',

In [15]:
df_forecats_columns = ['use [kW](t)', 'gen [kW](t)', 'Dishwasher [kW](t)', 'Furnace 1 [kW](t)',
       'Furnace 2 [kW](t)', 'Home office [kW](t)', 'Fridge [kW](t)',
       'Wine cellar [kW](t)', 'Garage door [kW](t)', 'Kitchen 12 [kW](t)',
       'Kitchen 14 [kW](t)', 'Kitchen 38 [kW](t)', 'Barn [kW](t)',
       'Well [kW](t)', 'Microwave [kW](t)', 'Living room [kW](t)',
       'Solar [kW](t)', 'temperature(t)', 'humidity(t)', 'visibility(t)',
       'apparentTemperature(t)', 'pressure(t)', 'windSpeed(t)',
       'windBearing(t)', 'precipIntensity(t)', 'dewPoint(t)',
       'precipProbability(t)']

In [16]:
def sliding_window_pca(data,n_windows,train_size,steps_ahead,transformation):

    result = {
         "window": [],
         "rmse": [],
         "mape": [],
         "mae": [],
         "r2": [],
         "smape": [],
         "nrmse": [],
         "variable":[]
    }
    
    final_result = {
         "window": [],
         "rmse": [],
         "mape": [],
         "mae": [],
         "r2": [],
         "smape": [],
         "nrmse": [],
         "variable":[]
    }
    
    num_fuzzy_rules = {
        "window": [],
        "qtd": []
    }

    tam = len(data)
    #n_windows = 30
    windows_length = math.floor(tam / n_windows)
    for ct, ttrain, ttest in Util.sliding_window(data, windows_length, train_size, inc=1):
        if len(ttest) > 0:
            
            print('-' * 20)
            print(f'training window {(ct)}')
            
            df_train = ttrain.loc[:,'use [kW](t-1)':'precipProbability(t-1)']
            df_test = ttest.loc[:,'use [kW](t-1)':'precipProbability(t-1)']
            df_original = ttest.loc[:,'use [kW](t)':'precipProbability(t)']
            
            models, data_train = mimo_kpca_nsfts.run_train(df_train,transformation)
            forecast, data_test = mimo_kpca_nsfts.run_test(models,df_test,steps_ahead,transformation)
            
            columns = list(df_forecats_columns)
            df_forecast = pd.DataFrame(forecast,columns=columns)
            
            #teste print fuzzy rules 
            sum_rules = 0
            for m in models:
                sum_rules = sum_rules + len(m)
                #print(len(m))
            
            #print(sum_rules)
            num_fuzzy_rules["window"].append(ct)
            num_fuzzy_rules["qtd"].append(sum_rules)
            
            for col in columns:  
                original = df_original[col].values
                forecast = df_forecast[col].values
#                 original = original[:len(original)-1]
#                 forecast = forecast[1:]
                
                #print("[{0: %H:%M:%S}]".format(datetime.datetime.now()) + f" getting statistics for variable: " + col)
                mae = round(mean_absolute_error(original,forecast),3)
                r2 = round(r2_score(original,forecast),3)
                #rmse = mean_squared_error(original,forecast,squared=False)
                rmse = round(Measures.rmse(original,forecast),3)
                mape = round(Measures.mape(original,forecast),3)
                nrmse = round(cal_nrmse(rmse, original),3)
                smape = round(Measures.smape(original,forecast),3)

                result["rmse"].append(rmse)
                result["nrmse"].append(nrmse)
                result["mape"].append(mape)
                result["mae"].append(mae)
                result["r2"].append(r2)
                result["smape"].append(smape)
                result["window"].append(ct)
                result["variable"].append(col)
                
#                 fig, ax = plt.subplots(nrows=1, ncols=1, figsize=[15, 3])
#                 ax.plot(original, label='Original')
#                 ax.plot(forecast, label='Forecast')
#                 handles, labels = ax.get_legend_handles_labels()
#                 lgd = ax.legend(handles, labels, loc=2, bbox_to_anchor=(1, 1))
#                 plt.show()
        
    measures = pd.DataFrame(result)
    rules = pd.DataFrame(num_fuzzy_rules)
    
    return measures, rules

## Sliding Window: MIMO KPCA-NSFTS

In [19]:
num_components_pca = 3
order_fts_model = 1
npart = 60
gamma = 0.1
fts_model = nsfts.NonStationaryFTS
memory_window_error = 3
steps_ahead = 1

mimo_kpca_nsfts = MimoNonStationaryFtsPca(num_components_pca, order_fts_model, npart,
                                            gamma, fts_model, memory_window_error)

In [20]:
kpca_result,rules =  sliding_window_pca(data=reframed,n_windows=30,train_size=0.75,steps_ahead=1,transformation='KPCA')

--------------------
training window 0
--------------------
training window 1679
--------------------
training window 3358
--------------------
training window 5037
--------------------
training window 6716
--------------------
training window 8395
--------------------
training window 10074
--------------------
training window 11753
--------------------
training window 13432
--------------------
training window 15111
--------------------
training window 16790
--------------------
training window 18469
--------------------
training window 20148
--------------------
training window 21827
--------------------
training window 23506
--------------------
training window 25185
--------------------
training window 26864
--------------------
training window 28543
--------------------
training window 30222
--------------------
training window 31901
--------------------
training window 33580
--------------------
training window 35259
--------------------
training window 36938
--------------------

In [21]:
columns = list(df_forecats_columns)

final_result = {
    "variable": [],
    "rmse": [],
    "mae": [],
    "mape": [],
    "r2": [],
    "smape": [],
    "nrmse": [],
    "rmse_std": [],
    "mae_std": [],
    "mape_std": [],
    "r2_std": [],
    "smape_std": [],
    "nrmse_std": []
}

kpca_result.to_csv (r'all_windows_kpca_mo_ensfts_kaggle_sh_10min.csv', index = False, header=True)

measures = kpca_result
var = measures.groupby("variable")

for col in columns:
    
    var_agr = var.get_group(col)
           
    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
    smape = round(statistics.mean(var_agr.loc[:,'smape']),3)
    nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)
    
    rmse_std = round(statistics.stdev(var_agr.loc[:,'rmse']),3)
    mape_std = round(statistics.stdev(var_agr.loc[:,'mape']),3)
    mae_std = round(statistics.stdev(var_agr.loc[:,'mae']),3)
    r2_std = round(statistics.stdev(var_agr.loc[:,'r2']),3)
    smape_std = round(statistics.stdev(var_agr.loc[:,'smape']),3)
    nrmse_std = round(statistics.stdev(var_agr.loc[:,'nrmse']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    final_result["mape"].append(mape)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
    final_result["smape"].append(smape)
    final_result["nrmse"].append(nrmse)
    
    final_result["rmse_std"].append(rmse_std)
    final_result["mape_std"].append(mape_std)
    final_result["mae_std"].append(mae_std)
    final_result["r2_std"].append(r2_std)
    final_result["smape_std"].append(smape_std)
    final_result["nrmse_std"].append(nrmse_std)
        
    #print(f'Results: {(col,rmse,mae,r2)}')
        
        
final_measures_kpca = pd.DataFrame(final_result) 



In [30]:
print("Statistics MIMO KPCA-NSFTS(test): ")
final_measures_kpca

Statistics MIMO KPCA-NSFTS(test): 


,variable,rmse,mae,mape,r2,smape,nrmse,rmse_std,mae_std,mape_std,r2_std,smape_std,nrmse_std
0,use [kW](t),0.837,0.485,250.845,-0.080,23.906,0.151,0.708,0.436,874.929,0.300,8.394,0.048
1,gen [kW](t),0.046,0.027,inf,0.813,48.941,0.103,0.017,0.012,NaN,0.130,10.165,0.034
2,Dishwasher [kW](t),0.171,0.033,inf,-5.089,94.502,0.126,0.045,0.013,NaN,28.429,2.735,0.033
3,Furnace 1 [kW](t),0.157,0.077,152.153,-0.466,27.481,0.221,0.081,0.064,135.975,0.464,19.314,0.120
4,Furnace 2 [kW](t),0.181,0.101,77.865,-0.484,19.868,0.303,0.127,0.096,70.127,0.695,17.005,0.169
5,Home office [kW](t),0.047,0.018,24.065,0.676,11.168,0.097,0.019,0.011,16.713,0.444,7.964,0.033
6,Fridge [kW](t),0.073,0.034,285.441,0.068,34.699,0.140,0.008,0.005,50.802,0.128,7.976,0.068
7,Wine cellar [kW](t),0.029,0.012,66.424,0.647,19.066,0.177,0.014,0.008,45.834,0.160,8.611,0.067
8,Garage door [kW](t),0.012,0.002,10.396,-0.857,3.364,0.092,0.013,0.002,6.454,0.353,1.791,0.024
9,Kitchen 12 [kW](t),0.009,0.001,inf,-0.155,19.839,0.017,0.022,0.004,NaN,0.443,20.103,0.032


In [23]:
#final_measures_kpca.to_csv (r'kpca_mo_ensfts_kaggle_sh_with_std.csv', index = False, header=True)

In [24]:
final_measures_kpca.to_csv (r'kpca_mo_ensfts_kaggle_sh_with_std_10min.csv', index = False, header=True)

In [25]:
avg_rules = round(statistics.mean(rules.loc[:,'qtd']),3)
avg_rules

169.8

## Sliding Window: MIMO PCA-NSFTS

In [26]:
num_components_pca = 8 #12 #8
order_fts_model = 1
npart = 60
gamma = 0.1
fts_model = nsfts.NonStationaryFTS
memory_window_error = 3
steps_ahead = 1

mimo_kpca_nsfts = MimoNonStationaryFtsPca(num_components_pca, order_fts_model, npart,
                                            gamma, fts_model, memory_window_error)

In [27]:
pca_result,rules =  sliding_window_pca(data=reframed,n_windows=30,train_size=0.75,steps_ahead=1,transformation='PCA')

--------------------
training window 0
--------------------
training window 1679
--------------------
training window 3358
--------------------
training window 5037
--------------------
training window 6716
--------------------
training window 8395
--------------------
training window 10074
--------------------
training window 11753
--------------------
training window 13432


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100


--------------------
training window 15111
--------------------
training window 16790


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100


--------------------
training window 18469
--------------------
training window 20148
--------------------
training window 21827
--------------------
training window 23506
--------------------
training window 25185
--------------------
training window 26864
--------------------
training window 28543
--------------------
training window 30222
--------------------
training window 31901
--------------------
training window 33580
--------------------
training window 35259


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100


--------------------
training window 36938
--------------------
training window 38617
--------------------
training window 40296
--------------------
training window 41975
--------------------
training window 43654
--------------------
training window 45333
--------------------
training window 47012
--------------------
training window 48691


In [29]:
columns = list(df_forecats_columns)

final_result = {
    "variable": [],
    "rmse": [],
    "mae": [],
    "mape": [],
    "r2": [],
    "smape": [],
    "nrmse": [],
    "rmse_std": [],
    "mae_std": [],
    "mape_std": [],
    "r2_std": [],
    "smape_std": [],
    "nrmse_std": []
}

pca_result.to_csv (r'all_windows_pca_mo_ensfts_kaggle_sh_10min.csv', index = False, header=True)

measures = pca_result
var = measures.groupby("variable")

for col in columns:
    
    var_agr = var.get_group(col)
           
    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
    smape = round(statistics.mean(var_agr.loc[:,'smape']),3)
    nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)

    rmse_std = round(statistics.stdev(var_agr.loc[:,'rmse']),3)
    mape_std = round(statistics.stdev(var_agr.loc[:,'mape']),3)
    mae_std = round(statistics.stdev(var_agr.loc[:,'mae']),3)
    r2_std = round(statistics.stdev(var_agr.loc[:,'r2']),3)
    smape_std = round(statistics.stdev(var_agr.loc[:,'smape']),3)
    nrmse_std = round(statistics.stdev(var_agr.loc[:,'nrmse']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    final_result["mape"].append(mape)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
    final_result["smape"].append(smape)
    final_result["nrmse"].append(nrmse)
    
    final_result["rmse_std"].append(rmse_std)
    final_result["mape_std"].append(mape_std)
    final_result["mae_std"].append(mae_std)
    final_result["r2_std"].append(r2_std)
    final_result["smape_std"].append(smape_std)
    final_result["nrmse_std"].append(nrmse_std)
        
    #print(f'Results: {(col,rmse,mae,r2)}')
        
        
final_measures_pca = pd.DataFrame(final_result) 

print("Statistics MIMO PCA-NSFTS(test): ")
final_measures_pca

Statistics MIMO PCA-NSFTS(test): 


,variable,rmse,mae,mape,r2,smape,nrmse,rmse_std,mae_std,mape_std,r2_std,smape_std,nrmse_std
0,use [kW](t),0.827,0.582,315.933,-0.077,30.742,0.150,0.668,0.504,942.190,0.278,11.109,0.042
1,gen [kW](t),0.067,0.049,inf,0.654,60.611,0.147,0.022,0.018,NaN,0.139,6.529,0.037
2,Dishwasher [kW](t),0.189,0.104,inf,-0.099,97.091,0.139,0.052,0.037,NaN,0.199,2.045,0.038
3,Furnace 1 [kW](t),0.142,0.098,222.662,-0.176,47.761,0.201,0.073,0.067,131.423,0.306,14.467,0.108
4,Furnace 2 [kW](t),0.158,0.115,104.351,-0.226,32.706,0.276,0.108,0.089,73.436,0.367,19.655,0.130
5,Home office [kW](t),0.090,0.063,98.624,0.087,35.138,0.198,0.040,0.035,63.562,0.815,15.384,0.084
6,Fridge [kW](t),0.082,0.061,607.431,-0.195,59.563,0.157,0.013,0.011,112.949,0.306,7.709,0.072
7,Wine cellar [kW](t),0.048,0.037,244.517,0.008,50.829,0.296,0.019,0.015,135.982,0.476,12.580,0.092
8,Garage door [kW](t),0.010,0.004,27.170,-0.249,12.348,0.075,0.011,0.005,32.695,0.279,11.353,0.022
9,Kitchen 12 [kW](t),0.009,0.005,inf,-0.298,29.178,0.025,0.020,0.010,NaN,0.522,30.878,0.055


In [22]:
#final_measures_pca.to_csv(r'pca_mo_ensfts_kaggle_sh_with_std.csv', index = False, header=True)

In [31]:
final_measures_pca.to_csv(r'pca_mo_ensfts_kaggle_sh_with_std_10min.csv', index = False, header=True)

In [32]:
#final_measures_pca.to_csv (r'pca_mo_ensfts_kaggle_sh_2.csv', index = False, header=True)

In [33]:
avg_rules = round(statistics.mean(rules.loc[:,'qtd']),3)
avg_rules

376.033